Objective:
- Show values over time
- Calc return

Input:

  - Trade (time, symbol, size, ->value)
  - Cash flow (time, amount, type -> inflow, outflow, tax, interest)
  - Price (time, symbol, qfq_price,bfq_price)
     - source: http://tushare.org/trading.html#id3

Processing:
  - Trade + Cash => Position
  - Position * price => Value
  - Value/value => return

Issues:
    - Price fq: stock split, dividend
    - Cash in/out: not affect performance    

Pyfolio data definition:

    returns : pd.Series
        Daily returns of the strategy, noncumulative.
         - Time series with decimal returns.
         - Example:
            2015-07-16    -0.012143
            2015-07-17    0.045350
            2015-07-20    0.030957
            2015-07-21    0.004902
    positions : pd.DataFrame, optional
        Daily net position values.
         - Time series of dollar amount invested in each position and cash.
         - Days where stocks are not held can be represented by 0 or NaN.
         - Non-working capital is labelled 'cash'
         - Example:
            index         'AAPL'         'MSFT'          cash
            2004-01-09    13939.3800     -14012.9930     711.5585
            2004-01-12    14492.6300     -14624.8700     27.1821
            2004-01-13    -13853.2800    13653.6400      -43.6375
    transactions : pd.DataFrame, optional
        Executed trade volumes and fill prices.
        - One row per trade.
        - Trades on different names that occur at the
          same time will have identical indicies.
        - Example:
            index                  amount   price    symbol
            2004-01-09 12:18:01    483      324.12   'AAPL'
            2004-01-09 12:18:01    122      83.10    'MSFT'
            2004-01-13 14:12:23    -75      340.43   'AAPL'
    market_data : pd.Panel, optional
        Panel with items axis of 'price' and 'volume' DataFrames.
        The major and minor axes should match those of the
        the passed positions DataFrame (same dates and symbols).


In [1]:
import pandas as pd
import numpy as np
import pyfolio as pf
import matplotlib.pyplot as plt
import tushare as ts

ModuleNotFoundError: No module named 'pyfolio'

In [2]:

longer_ts = pd.Series(np.random.randn(10),
                      index=pd.date_range('1/1/2000', periods=10))
longer_ts
# longer_ts['2001']

2000-01-01    0.301875
2000-01-02   -0.073903
2000-01-03    0.578376
2000-01-04   -0.623029
2000-01-05   -0.474536
2000-01-06    0.624435
2000-01-07   -0.756235
2000-01-08    0.888046
2000-01-09   -0.444444
2000-01-10   -0.026610
Freq: D, dtype: float64

In [3]:
longer_ts['2000-01-01']

0.30187492918826114

In [2]:
txn=pd.read_csv('txn.csv',sep=',',parse_dates=[0],\
                skiprows=0,dtype={'time':str,'amount':np.int16,'price':np.float,'symbol':str},\
                index_col=[0])

In [3]:
txn

,amount,price,symbol
index,,,
2018-07-17 12:18:01,100,750.0,600519
2019-04-01 14:12:23,-100,860.0,600519


In [4]:
txn.index

DatetimeIndex(['2018-07-17 12:18:01', '2019-04-01 14:12:23'], dtype='datetime64[ns]', name='index', freq=None)

In [5]:
txn.dtypes

amount      int16
price     float64
symbol     object
dtype: object

In [12]:
price=pd.read_csv('MKTDATA.csv',sep='\s+',parse_dates=[0],skiprows=0,\
                  dtype={'date':str, 'high':float,'open':float,'close':float,\
                       'low':float,'volume':float,'amount':float})

In [13]:
price.index

RangeIndex(start=0, stop=10, step=1)

In [14]:
price.head(10)

,date,open,high,close,low,volume,amount
0,2015-03-16,13.27,13.45,13.39,13.00,81212976.0,1.073863e+09
1,2015-03-13,13.04,13.38,13.37,13.00,40548836.0,5.327397e+08
2,2015-03-12,13.29,13.95,13.28,12.96,71505720.0,9.629799e+08
3,2015-03-11,13.35,13.48,13.15,13.00,59110248.0,7.803007e+08
4,2015-03-10,13.16,13.67,13.59,12.72,105753088.0,1.393820e+09
5,2015-03-09,13.77,14.73,14.13,13.70,139091552.0,1.994455e+09
6,2015-03-06,12.17,13.39,13.39,12.17,89486704.0,1.167753e+09
7,2015-03-05,12.79,12.80,12.17,12.08,26040832.0,9.669274e+08
8,2015-03-04,13.96,13.96,13.30,12.58,26636174.0,1.060271e+09
9,2015-03-03,12.17,13.10,13.10,12.05,19290366.0,7.333368e+08


In [15]:
price.dtypes

date      datetime64[ns]
open             float64
high             float64
close            float64
low              float64
volume           float64
amount           float64
dtype: object

In [16]:
Price=ts.get_h_data('600690',start='2015-01-01',end='2015-09-01',retry_count=10)

NameError: name 'ts' is not defined

In [17]:
Price

NameError: name 'Price' is not defined

In [13]:
try:
    Price=ts.get_h_data('600690', start='2013-01-01', end='2015-03-16') #两个日期之间的前复权数据
except:
    print('sth wrong')

[Getting data:]#

In [14]:
Price['symbol']='600690'

In [15]:
Price.to_csv('mktdata600690.csv')

In [22]:
Price=pd.read_csv("600690.csv", sep="\s+",parse_dates=[0],skiprows=0,\
                  dtype={'date':str, 'high':float,'open':float,'close':float,\
                       'low':float,'volume':float,'code':str})

In [25]:
Price.tail()

,date,open,close,high,low,volume,code
3647,2018-08-28,15.50,15.31,15.58,15.22,287737.0,600690
3648,2018-08-29,15.25,15.05,15.30,15.00,220981.0,600690
3649,2018-08-30,15.05,15.15,15.27,15.01,230191.0,600690
3650,2018-08-31,14.87,14.89,14.94,14.56,345620.0,600690
3651,2018-09-03,14.76,14.89,14.94,14.66,175623.0,600690


In [24]:
Price.dtypes

date      datetime64[ns]
open             float64
close            float64
high             float64
low              float64
volume           float64
code              object
dtype: object

for each symbol
    get initial offer date
    for every 3 years:
        get price

In [4]:
symbols=['000001','000651','000858','002001',\
         '002511','600036','600519','600518','600690','600789']

In [19]:
def get_price(symbols,start,end):
    if (start>end):
        raise 'Start cannot greater than end!'
        return None
    for symbol in symbols:
        if (start+3>end):
            price=ts.get_hist_data(symbol,start=str(start)+'-01-01',end=str(start+3)+'-01-01')
            price['symbol']=symbol
            price.to_csv('price.csv',mode='a+')
        else:
            i=start
            while (i+3<end):
                price=ts.get_hist_data(symbol,start=str(i)+'-01-01',end=str(i+3)+'-01-01')
                price['symbol']=symbol
                price.to_csv('price.csv',mode='a+')
                i=i+3
        

In [20]:
get_price(symbols,2010,2020)

In [52]:
#mktdata=pd.read_csv('price.csv',sep=',',skiprows=2)
mktdata=pd.read_csv?

In [82]:
mktdata=pd.read_csv('price.csv',sep=',',parse_dates=[0],skiprows=2,\
                    names=('date','open','high','close','low','volume','price_change','p_change','ma5',\
                                'ma10','ma20','v_ma5','v_ma10','v_ma20','symbol'),\
                  dtype={'high':float,'open':float,'close':float,\
                       'low':float,'volume':float,'symbol':str}) #,
                   #index_col=[0])

In [89]:
mktdata.head()

,,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
date,symbol,,,,,,,,,,,,,
2018-12-28,000001,9.31,9.46,9.38,9.31,576604.00,0.10,1.08,9.344,9.623,9.980,543614.19,641599.88,599667.24
2018-12-27,000001,9.45,9.49,9.28,9.28,624593.25,-0.02,-0.21,9.358,9.702,10.029,628416.76,636606.99,600572.81
2018-12-26,000001,9.35,9.42,9.30,9.27,421140.59,-0.04,-0.43,9.444,9.813,10.075,701555.07,653232.18,593607.35
2018-12-25,000001,9.29,9.43,9.34,9.21,586615.44,-0.08,-0.85,9.572,9.907,10.124,736928.35,647756.03,599437.89
2018-12-24,000001,9.40,9.45,9.42,9.31,509117.69,-0.03,-0.32,9.728,9.996,10.166,727154.12,619003.45,606779.54


In [88]:
mktdata.set_index(['date','symbol'],inplace=True)

In [15]:
# mktdata=mktdata.iloc[mktdata.symbol == np.any(symbols),:]

In [74]:
txn

,index,amount,price,symbol
0,2018-07-17 12:18:01,100,750.0,'600519'
1,2019-04-01 14:12:23,-100,860.0,'600519'


In [5]:
positions=pd.DataFrame(index=pd.date_range('9/1/2016', periods=1500),columns=["cash"])

In [6]:
positions.tail(10)

,cash
2020-09-30,NaN
2020-10-01,NaN
2020-10-02,NaN
2020-10-03,NaN
2020-10-04,NaN
2020-10-05,NaN
2020-10-06,NaN
2020-10-07,NaN
2020-10-08,NaN
2020-10-09,NaN


In [7]:
positions['cash']=1000000.

In [8]:
for symbol in symbols:
    positions[symbol+'_shares']=0.
    positions[symbol+'_value']=0.

In [9]:
positions.tail()

,cash,000001_shares,000001_value,000651_shares,000651_value,000858_shares,000858_value,002001_shares,002001_value,002511_shares,...,600036_shares,600036_value,600519_shares,600519_value,600518_shares,600518_value,600690_shares,600690_value,600789_shares,600789_value
2020-10-05,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-06,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-07,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-08,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-09,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


issue - intra-day transaction

In [15]:
txn.iloc[0].index

Index(['amount', 'price', 'symbol'], dtype='object')

In [10]:
positions.iloc[0]['cash']

1000000.0

In [17]:
txn.iloc[t][0]

100

In [21]:
txn.index

DatetimeIndex(['2018-07-17 12:18:01', '2019-04-01 14:12:23'], dtype='datetime64[ns]', name='index', freq=None)

In [23]:
positions.index

DatetimeIndex(['2016-09-01', '2016-09-02', '2016-09-03', '2016-09-04',
               '2016-09-05', '2016-09-06', '2016-09-07', '2016-09-08',
               '2016-09-09', '2016-09-10',
               ...
               '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03',
               '2020-10-04', '2020-10-05', '2020-10-06', '2020-10-07',
               '2020-10-08', '2020-10-09'],
              dtype='datetime64[ns]', length=1500, freq='D')

In [11]:
for td in txn.index:
    print(txn['symbol'][td])
    for d in positions.index:
          if d>td:
                sbl=txn['symbol'][td]
                positions[sbl+'_shares'][d] += txn['amount'][td]
                positions[sbl+'_value'][d] += txn['amount'][td]*txn['price'][td]
                positions['cash'][d] -= txn['amount'][td]*txn['price'][td]

600519
600519


In [12]:
positions.tail()

,cash,000001_shares,000001_value,000651_shares,000651_value,000858_shares,000858_value,002001_shares,002001_value,002511_shares,...,600036_shares,600036_value,600519_shares,600519_value,600518_shares,600518_value,600690_shares,600690_value,600789_shares,600789_value
2020-10-05,1011000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-11000.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-06,1011000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-11000.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-07,1011000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-11000.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-08,1011000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-11000.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-09,1011000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-11000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
positions.loc['2018-07-17':'2018-07-18']

,cash,000001_shares,000001_value,000651_shares,000651_value,000858_shares,000858_value,002001_shares,002001_value,002511_shares,...,600036_shares,600036_value,600519_shares,600519_value,600518_shares,600518_value,600690_shares,600690_value,600789_shares,600789_value
2018-07-17,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-18,925000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,100.0,75000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
positions.loc['2019-04-01':'2019-04-02']

,cash,000001_shares,000001_value,000651_shares,000651_value,000858_shares,000858_value,002001_shares,002001_value,002511_shares,...,600036_shares,600036_value,600519_shares,600519_value,600518_shares,600518_value,600690_shares,600690_value,600789_shares,600789_value
2019-04-01,925000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,100.0,75000.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-02,1011000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-11000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
for d in positions.index:
    for col in positions.columns:
        if "_value" in col:
            symbol=col[0:6]
            positions[col][d]=float(positions[symbol+"_shares"][d]) * mktdata.loc[d,symbol]['close'][0]

d:\softbak\anaconda3\lib\site-packages\ipykernel_launcher.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  """


KeyError: Timestamp('2016-09-01 00:00:00', freq='D')

In [65]:
"000_value"[0:2]

'00'

In [98]:
mktdata.loc['2018-09-05','600518']['close'][0]

d:\softbak\anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


20.49